# 딥러닝을 사용한 추천 시스템

## Matrix Factorization MF 신경망으로 변환하기

In [3]:
# csv 파일에서 불러오기
import pandas as pd

from sklearn.model_selection import train_test_split
import tensorflow as tf 
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Add, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD, Adamax


base_src = ''
u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src, sep='\t',
                      names=r_cols, encoding='latin-1')

ratings_train, ratings_test = train_test_split(
    ratings,
    test_size=0.2,
    shuffle=True,
    random_state=2022
)


In [15]:
K = 200

mu = ratings_train.rating.mean()

M = ratings.user_id.max() + 1
N = ratings.movie_id.max() + 1

def RMSE(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true-y_pred)))
    

In [16]:
user = Input(shape=(1,))
item = Input(shape=(1,))

P_embedding = Embedding(M, K, embeddings_regularizer=l2())(user)
Q_embedding = Embedding(N, K, embeddings_regularizer=l2())(item)

user_bias = Embedding(M, 1, embeddings_regularizer=l2())(user)
item_bias = Embedding(M, 1, embeddings_regularizer=l2())(user)

In [30]:
R = layers.dot([P_embedding, Q_embedding], axes=(2, 2))
R = layers.add([R, user_bias, item_bias])
R = Flatten()(R)

model = Model(inputs=[user, item], outputs=R)
model.compile(
    loss=RMSE,
    optimizer=SGD(),
    metrics=[RMSE]
)

model.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 1, 200)       188800      ['input_3[0][0]']                
                                                                                                  
 embedding_5 (Embedding)        (None, 1, 200)       336600      ['input_4[0][0]']                
                                                                                           

In [34]:
result = model.fit(
    x = [ratings_train.user_id.values,
         ratings_train.movie_id.values],
    y = ratings_train.rating.values - mu,
    epochs = 60,
    batch_size = 256,
    validation_data = (
        [ratings_test.user_id.values,
            ratings_test.movie_id.values],
        ratings_test.rating.values - mu
    )
)

Epoch 1/60
313/313 [==============================] - 1s 4ms/step - loss: 2.6320 - RMSE: 1.1106 - val_loss: 2.5364 - val_RMSE: 1.1069
Epoch 2/60
313/313 [==============================] - 1s 3ms/step - loss: 2.4527 - RMSE: 1.1096 - val_loss: 2.3682 - val_RMSE: 1.1059
Epoch 3/60
313/313 [==============================] - 1s 3ms/step - loss: 2.2945 - RMSE: 1.1084 - val_loss: 2.2198 - val_RMSE: 1.1050
Epoch 4/60
313/313 [==============================] - 1s 3ms/step - loss: 2.1548 - RMSE: 1.1074 - val_loss: 2.0889 - val_RMSE: 1.1042
Epoch 5/60
313/313 [==============================] - 1s 4ms/step - loss: 2.0316 - RMSE: 1.1066 - val_loss: 1.9734 - val_RMSE: 1.1034
Epoch 6/60
313/313 [==============================] - 1s 3ms/step - loss: 1.9231 - RMSE: 1.1059 - val_loss: 1.8715 - val_RMSE: 1.1027
Epoch 7/60
313/313 [==============================] - 1s 3ms/step - loss: 1.8273 - RMSE: 1.1052 - val_loss: 1.7816 - val_RMSE: 1.1021
Epoch 8/60
313/313 [==============================] - 1s 3ms/s